# Custom Model UPI Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to create and deploy custom model which using IRIS classifier based on xgboost model into Merlin.

In [ ]:
!python3 -m pip install --upgrade -r requirements.txt > /dev/null

In [ ]:
import merlin
import warnings
import os
import xgboost as xgb
from merlin.model import ModelType
from sklearn.datasets import load_iris
warnings.filterwarnings('ignore')


## 1. Initialize Merlin Resources


### 1.1 Set Merlin Server

In [ ]:
# Set Merlin Server
merlin.set_url("http://localhost:8080")

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in Merlin is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by Merlin are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("custom-model-upi", ModelType.CUSTOM)

## 2. Deploy Model

### 2.2 Create Model Version and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_custom_model(image="ghcr.io/gojek/custom-model:v0.3",model_dir=model_dir)
v.finish()
```


This image `ghcr.io/tiopramayudi/custom-predictor-go:v0.2` is built by using this [Dockerfile](./Dockerfile). The image contains go web service executable where the code you can find [here](./)

In [ ]:
# Create new version of the model
with merlin.new_model_version() as v:
    # Upload the serialized model to Merlin
    merlin.log_custom_model(image="ghcr.io/tiopramayudi/custom-model-upi:v0.1")
    

### 2.2 Deploy Model


In [ ]:
from merlin.protocol import Protocol

endpoint = merlin.deploy(v, protocol = Protocol.UPI_V1)

### 2.3 Send Test Request

In [ ]:
from caraml.upi.v1 import upi_pb2, upi_pb2_grpc
from caraml.upi.v1 import type_pb2, variable_pb2
import grpc
import pandas as pd
from caraml.upi.utils import df_to_table, table_to_df

def create_simple_forwarder_request() -> upi_pb2.PredictValuesRequest:
    target_name = "probability"
    cols = ["id", "name", "vehicle", "previous_vehicle", "rating", "test_time", "row_number"]
    indices = ["row1", "row2"]
    rows = [
        [1, "driver-1", "motorcycle", "suv", 4.0, 90, 0],
        [2, "driver-2", "sedan", "mpv", 3.0, 90, 1]
    ]
    driver_df = pd.DataFrame(columns=cols, data=rows, index=indices)
    driver_table = df_to_table(driver_df, "driver_table")
    return upi_pb2.PredictValuesRequest(
        target_name=target_name,
        prediction_table=driver_table,
    )

def simple_forwarder_response(request:upi_pb2.PredictValuesRequest) -> upi_pb2.PredictValuesResponse:
    return upi_pb2.PredictValuesResponse(
        target_name=request.target_name,
        prediction_result_table=request.prediction_table
    )

channel = grpc.insecure_channel(f"{endpoint.url}:80")
stub = upi_pb2_grpc.UniversalPredictionServiceStub(channel)


request = create_simple_forwarder_request()
response = stub.PredictValues(request=request)

assert response.metadata.prediction_id == request.metadata.prediction_id
assert response.target_name == request.target_name
exp_response = simple_forwarder_response(request)
print(request)
assert response == exp_response


### 2.4 Delete Deployment

In [ ]:
merlin.undeploy(v)
